In [11]:
import os
import cv2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import  Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from tensorflow.keras.metrics import Recall

In [12]:
train_dir = 'fire_data/train'
valid_dir = 'fire_data/valid'
test_dir = 'fire_data/test'

model_path = 'wildfire_detection_model.keras'

In [13]:
def load_data(directory, img_size=(32, 32)):
    X, Y = [], []
    for direct in os.listdir(directory):
        direct_path = os.path.join(directory, direct)
        if not os.path.isdir(direct_path):
            continue
        for filename in os.listdir(direct_path):
            img_path = os.path.join(direct_path, filename)
            img = cv2.imread(img_path)
            if img is None:
                continue
            img = cv2.resize(img, img_size)
            img = img.astype(np.float32) / 255.0
            X.append(img)
            Y.append(direct)
    return np.array(X), np.array(Y)

x_train, y_train = load_data(train_dir)
x_val, y_val = load_data(valid_dir)
x_test, y_test = load_data(test_dir)

# Convert labels to binary
y_train = np.array([1 if label == 'wildfire' else 0 for label in y_train])
y_val = np.array([1 if label == 'wildfire' else 0 for label in y_val])
y_test = np.array([1 if label == 'wildfire' else 0 for label in y_test])

print("x_train shape:", x_train.shape)
print("x_val shape:", x_val.shape)
print("x_test shape:", x_test.shape)
print("y_train shape:", y_train.shape)
print("y_val shape:", y_val.shape)
print("y_test shape:", y_test.shape)

Premature end of JPEG file
Premature end of JPEG file


x_train shape: (30250, 32, 32, 3)
x_val shape: (6300, 32, 32, 3)
x_test shape: (6300, 32, 32, 3)
y_train shape: (30250,)
y_val shape: (6300,)
y_test shape: (6300,)


In [14]:
# Define the model
model = Sequential([
    Input(shape=(32, 32, 3)),  # Specify the input shape here
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.1),
    Dense(1, activation='sigmoid')  # Binary classification
])

In [15]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_2 (Conv2D)               │ (None, 30, 30, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 15, 15, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 13, 13, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 6, 6, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 2304)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │       295,040 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 314,561 (1.20 MB)

 Trainable params: 314,561 (1.20 MB)

 Non-trainable params: 0 (0.00 B)

In [16]:
# Compile the model
optimizer = Adam(learning_rate=0.0008396)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=[Recall(name='val_recall')])

# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model
history = model.fit(
    x_train, y_train,
    validation_data=(x_val, y_val),
    epochs=40,
    batch_size=32,
    callbacks=[early_stopping]
)

# Save the model
model.save(model_path)

Epoch 1/40
946/946 ━━━━━━━━━━━━━━━━━━━━ 11s 11ms/step - loss: 0.3089 - val_recall: 0.8465 - val_loss: 0.1593 - val_val_recall: 0.9454
Epoch 2/40
946/946 ━━━━━━━━━━━━━━━━━━━━ 8s 9ms/step - loss: 0.1983 - val_recall: 0.9376 - val_loss: 0.1653 - val_val_recall: 0.9109
Epoch 3/40
946/946 ━━━━━━━━━━━━━━━━━━━━ 8s 9ms/step - loss: 0.1652 - val_recall: 0.9485 - val_loss: 0.2068 - val_val_recall: 0.8747
Epoch 4/40
946/946 ━━━━━━━━━━━━━━━━━━━━ 8s 9ms/step - loss: 0.1591 - val_recall: 0.9503 - val_loss: 0.1523 - val_val_recall: 0.9287
Epoch 5/40
946/946 ━━━━━━━━━━━━━━━━━━━━ 8s 9ms/step - loss: 0.1493 - val_recall: 0.9574 - val_loss: 0.1291 - val_val_recall: 0.9609
Epoch 6/40
946/946 ━━━━━━━━━━━━━━━━━━━━ 8s 9ms/step - loss: 0.1387 - val_recall: 0.9578 - val_loss: 0.1334 - val_val_recall: 0.9647
Epoch 7/40
946/946 ━━━━━━━━━━━━━━━━━━━━ 8s 9ms/step - loss: 0.1304 - val_recall: 0.9616 - val_loss: 0.1372 - val_val_recall: 0.9578
Epoch 8/40
946/946 ━━━━━━━━━━━━━━━━━━━━ 9s 9ms/step - loss: 0.1177 - val_r

In [17]:
# Predict on test data
test_predictions = model.predict(x_test)
y_pred = (test_predictions > 0.45).astype("int32").ravel()

# Evaluate with sklearn metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='binary')

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

197/197 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
Accuracy: 0.9596825396825397
Precision: 0.9653779572994806
Recall: 0.9614942528735633
F1 Score: 0.9634321911891737


In [18]:
report = classification_report(y_test, y_pred, target_names=["Class 0", "Class 1"])

print("\nClassification Report:\n")
print(report)



Classification Report:

              precision    recall  f1-score   support

     Class 0       0.95      0.96      0.96      2820
     Class 1       0.97      0.96      0.96      3480

    accuracy                           0.96      6300
   macro avg       0.96      0.96      0.96      6300
weighted avg       0.96      0.96      0.96      6300

